In [1]:
!pip install yfinance tensorflow scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 15.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 17.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for peewee: filename=peewee-3.18.1-cp312-cp312-macosx_11_0_arm64.whl size=264312 sha256=7394250aee7937d3e3e9388747aa4729ec3c56b62d25c731371ccb167176b593
  Stored in directory: /Users/khandil/Library/Caches/pip/wheels/1a/57/6a/bb71346381d0d911cd4ce3026f1fa720da76707e4f01cf27dd
Successfully built peewee


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 1: Download Stock Data
data = yf.download("AAPL", start="2010-01-01", end="2023-11-13")
df = data[['Close']].reset_index()

# Step 2: Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close']])

# Step 3: Create Sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

# Step 4: Split into training and test sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Step 5: Reshape for LSTM
X_train = X_train.reshape(-1, seq_length, 1)
X_test = X_test.reshape(-1, seq_length, 1)

# Step 6: Build the Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(LSTM(50))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 7: Train the Model
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Step 8: Make Predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Step 9: Show Error
rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
print(f"RMSE: {rmse}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Epoch 1/5



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0065
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.4950e-04
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.2657e-04
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.2138e-04
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.2244e-04
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
RMSE: 5.7447777263490405
